# Description

Notebook to compute metrics.

# Setup

In [1]:
%matplotlib notebook
%load_ext autoreload

## Imports

In [2]:
# os related
from pathlib import Path
import os 

# data related
import pandas as pd
import missingno as msno
import datetime as dt
import numpy as np


# visual related
import matplotlib.pyplot as plt
# time related

In [3]:

from fonctions import *


## Paths

In [4]:
# MUST BE IN CODE FOLDER TO WORK !
PROJECT_PATH = Path(os.getcwd()).parent

RAW_DATA = PROJECT_PATH / "RAW_DATA" 
PROC_DATA = PROJECT_PATH / "PROC_DATA"

# RAW FILES
CSV_FILE = RAW_DATA / "Individual_stock_data_csv.csv"
XLSX_FILE = RAW_DATA / "Individual_stock_data.xlsx"

# PROC FILES
PROC_STOCK = PROC_DATA / "proc_stock.csv"
PROC_CARBON = PROC_DATA / "proc_carbon.csv"

STOCK_2000 = PROC_DATA / "stock_2000.csv"

STOCK_WORK = PROC_DATA / "stock_work.csv"
STOCK2USE = PROC_DATA / "stock2use.csv"
DIC2USE = PROC_DATA / "dic2use.json"


OUT_RENDEMENT = PROC_DATA / "rendement_v1.csv"

## Utils

In [6]:

from fonctions import *

# Notebook

In [5]:
return_df = pd.read_csv(OUT_RENDEMENT)
return_df.index = read_file(STOCK2USE).index[104:]
return_df.drop(columns="Unnamed: 0", inplace=True)

In [6]:

sp500 = read_file(STOCK_WORK)
bm_return = equally_weighted(sp500)


In [27]:
return_sp = (sp500.mean(axis=1).pct_change())*100

In [24]:
(sp500.pct_change()).mean(axis=1)*100

timestamp
1999-12-14         NaN
1999-12-21    1.621024
1999-12-28    2.917997
2000-01-04   -2.224484
2000-01-11    4.114965
                ...   
2019-11-12    0.216510
2019-11-19    0.577101
2019-11-26    0.851971
2019-12-03   -1.712872
2019-12-10    1.596631
Length: 1044, dtype: float64

In [9]:
metrics = ["ret", "vol", "sr", "track_err", "ex_ret", "enc", "encb", "ir", "di"]

In [12]:
rf = 0.5

In [13]:
dic = dict.fromkeys(return_df.columns)

for k in dic.keys():
    dic[k] = dict.fromkeys(metrics)

timestamp
1999-12-14         NaN
1999-12-21    1.981972
1999-12-28    2.785832
2000-01-04   -3.240800
2000-01-11    4.855155
                ...   
2019-11-12    0.003154
2019-11-19    0.239794
2019-11-26    0.641848
2019-12-03   -2.530275
2019-12-10    0.783231
Length: 1044, dtype: float64

In [48]:
for pf in dic.keys():
        
    # Rendement annualisé
    dic[pf]["ret"] = return_df[pf].mean() * 52

    # Volatilité sur les 18 ans
    dic[pf]["vol"] = return_df[pf].std() * np.sqrt(52)
    
    # Ratio de sharp
    dic[pf]["sr"] = (dic[pf]["ret"] - rf) / dic[pf]["vol"]
    
    # Tracking error
    dic[pf]["track_err"] = 
    
    # Excess return 
    dic[pf]["ex_ret"] = 
    
    # Calcul de l'ENC
    # dic[pf]["enc"]
    
    # Information ratio
    # dic[pf]["ir"]
    
    # ENCB
    # dic[pf]["encb"]
    
    # Diversification index
    # dic[pf]["di"]